In [1]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import dash_table

In [2]:
df = pd.read_csv("../NBA_INTL_FIBA_Clustering.csv", encoding='latin-1', index_col=0)
all_names = ['Average'] + df.index.tolist()

# Split up into separate dataframes
FIBA_df = df[[c for c in df.columns if c.startswith('FIBA')]].dropna()
NBA_df = df[[c for c in df.columns if c.startswith('NBA')]].dropna()
INTL_df = df[[c for c in df.columns if c.startswith('INTL')]].dropna()
dfs = [FIBA_df, NBA_df, INTL_df]
for d in dfs:
    print(d.shape)

(1306, 11)
(986, 11)
(3871, 11)


In [3]:
# Normalize dataframes, add row for 'Average', put cluster series in list
cluster_series = []
new_col_names = ['BH', 'SU', 'TR', 'IS', 'PU', 'RM', 'CT', 'OR', 'OS', 'HO']
colours = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

for i in range(len(dfs)):
    cluster_series.append(dfs[i].iloc[:, -1])
    df = dfs[i].iloc[:, :-1]
    if i == 0:
        original_col_names = [c[5:] for c in df.columns]
    df.columns = new_col_names
    #df = df.rename(columns={c: c[5:] for c in df.columns})
    
    df = df.div(df.sum(axis=1), axis=0) # Normalize by row
    mean_vals = df.mean().rename("Average")
    dfs[i] = pd.concat([df, pd.DataFrame(mean_vals).transpose()])

col_map = [(new_col_names[i] + ':\t' + original_col_names[i]) for i in range(len(original_col_names))]
for d in dfs:
    print(d.shape)

(1307, 10)
(987, 10)
(3872, 10)


In [4]:
clusters_df = pd.concat(cluster_series, axis=1)
clusters_df = clusters_df.reset_index().rename(columns={'index': 'Player'})
labels = ['Stretch bigs', 'Playmakers', 'Ball handlers', 'Driving bigs', 'Shooters', 'Low-post bigs', 'Off-ball wings']
labels = [labels[i] + ' (' + str(i) + ')' for i in range(len(labels))]
clusters_df

,Player,FIBA_Cluster,NBA_Cluster,INTL_Cluster
0,Bogdan Bogdanovic,1.0,1.0,1.0
1,Luis Scola,5.0,0.0,5.0
2,Facundo Campazzo,2.0,NaN,2.0
3,Mantas Kalnietis,2.0,NaN,2.0
4,Bojan Bogdanovic,6.0,6.0,1.0
...,...,...,...,...
5239,Nobel Boungou Colo,NaN,NaN,6.0
5240,Davis Geks,NaN,NaN,6.0
5241,Arizona Reid,NaN,NaN,5.0
5242,Jimmy Gavin,NaN,NaN,2.0


In [37]:
cluster_num = 3
dfs_by_cluster = []
for i in range(7):
    dfs_by_cluster.append(pd.DataFrame())
    for j in range(3):
        s = (cluster_series[j] == i)
        dfs_by_cluster[i] = pd.concat([dfs_by_cluster[i], dfs[j].loc[s.index[s]]])
    dfs_by_cluster[i] = dfs_by_cluster[i].mean()
df_by_cluster = pd.concat(dfs_by_cluster, axis=1)
df_by_cluster

,0,1,2,3,4,5,6
BH,0.019403,0.419207,0.595497,0.002456,0.057168,0.002820,0.216384
SU,0.259131,0.188883,0.128285,0.073625,0.411540,0.073507,0.275954
TR,0.105021,0.127928,0.090012,0.070276,0.137154,0.055147,0.160798
IS,0.051340,0.090431,0.086280,0.017541,0.036200,0.024284,0.067466
PU,0.169476,0.016494,0.005910,0.157157,0.046842,0.357566,0.028341
RM,0.110060,0.001256,0.000381,0.215006,0.042102,0.152968,0.005115
CT,0.112502,0.027349,0.013233,0.214083,0.078013,0.152379,0.047195
OR,0.136909,0.045000,0.029437,0.243829,0.116625,0.172986,0.069168
OS,0.029205,0.051027,0.023631,0.004004,0.056208,0.006816,0.090220
HO,0.006952,0.032425,0.027332,0.002023,0.018149,0.001527,0.039359


In [5]:
def get_polar_plots_league(df, players):
    """
    df: A dataframe with the play type distribution for each player in a particular league
    players: A list of players to look up, possibly including 'Average'
    If a player is not in the dataframe, does not plot this player
    """
    global colours
    
    if not players: # No players passed
        return px.line_polar(r = [0 for i in range(10)], theta = df.columns)
    
    valid_players = df.index.intersection(players)
    
    gs = []
    for i in range(len(players)):
        p = players[i]
        if p in valid_players:
            r = df.loc[p].tolist()
            theta = df.columns.tolist()
            g = go.Scatterpolar(r = r + [r[0]], theta = theta + [theta[0]], mode = 'lines', name = p,
                               line = {'color': colours[i%10]}, legendgroup = i, showlegend = (i not in existing_groups))
            existing_groups.add(i)
            gs.append(g)

    return tuple(gs)

In [67]:
def get_polar_plot_cluster(cluster):
    """
    cluster: the cluster number
    """
    r = df_by_cluster[cluster].tolist()
    theta = df_by_cluster.index.tolist()
    g = go.Scatterpolar(r = r + [r[0]], theta = theta + [theta[0]], mode = 'lines',
                        name = labels[cluster], line = {'color': colours[cluster]}, legendgroup = cluster,
                       )
    return g

cluster_fig = px.scatter_polar()
for i in range(7):
    cluster_fig.add_trace((get_polar_plot_cluster(i)))
cluster_fig.update_layout(polar_angularaxis_rotation=0, polar_angularaxis_direction='counterclockwise')

cluster_fig.show()

In [68]:
dropdown_options = [{"label":i, "value": i} for i in all_names]

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("Play Type Distribution of Players"),
    html.P("Select players to display from the dropdown. Selecting 'Average' shows the mean play type distribution for all players in that competition."),
    html.P("Click on a player in the legend below to hide/show that series. Zoom in by clicking and dragging, and reset the zoom by double-clicking."),
    dcc.Dropdown(id="selected_players", options=dropdown_options, value="Average", multi=True, placeholder="Search for a player or 'Average'",),
    dcc.Graph(id='play_type_dist_by_player'),
    html.P('\n'.join(col_map), style={'whiteSpace': 'pre-wrap'}),
    dash_table.DataTable(id='player_clusters', columns=[{"name": i, "id": i} for i in clusters_df.columns],
                        style_header={'fontWeight': 'bold'}, sort_action='native'),
    dcc.Graph(id='play_type_dist_by_cluster', figure=cluster_fig)])
    
    
@app.callback(Output('play_type_dist_by_player', 'figure'), [Input("selected_players", "value")])
def update_figure(players):
    global existing_groups
    existing_groups = set()
    if isinstance(players, str):
        players = [players]
    fig = make_subplots(cols=3, specs=[[{"type": "polar"}, {"type": "polar"}, {"type": "polar"}]],
                       subplot_titles=("<b>FIBA</b>", "<b>NBA</b>", "<b>INTL</b>"))
    for i in range(3):
        for g in get_polar_plots_league(dfs[i], players, showlegend = (i==0)):
            fig.add_trace(g, row=1, col=i+1)
            fig.layout.annotations[i].update(y=1.05)
    
    fig.update_polars(radialaxis_range=[0, 0.85])
    fig.update_layout(legend = {'orientation': "h", 'yanchor': "bottom", 'y': -0.15}, margin = {'l': 0, 'r': 0, 't': 50})
    
    return fig

@app.callback(Output('player_clusters', 'data'), [Input("selected_players", "value")])
def display_clusters(players):
    if isinstance(players, str):
        players = [players]
    ps = [p for p in players if p != 'Average']
    d = clusters_df[clusters_df['Player'].isin(ps)].fillna(' N/A')
    return d.replace([i for i in range(7)], [l for l in labels]).to_dict('records')

app.run_server(port=8056)
#app.run_server(mode="inline", debug=True, port=8056)

Dash app running on http://127.0.0.1:8056/
